<a href="https://colab.research.google.com/github/NastasiaMazur/Finance-Sentiment-Analysis/blob/main/albert_xlarge_v2_FinanceIncauditor_sentiment_UPDATED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downstream Task: Sentiment Analysis

Model: **albert-xlarge-v2**

Dataset: **FinanceInc/auditor_sentiment**

# 1. Activate GPU and Install Dependencies

In [1]:
#check if GPU is available
import torch
torch.cuda.is_available()

True

In [2]:
# Install required libraries
!pip install transformers
!pip install datasets
!pip install bertviz transformers
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 29.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-no

In [3]:
# Conntext Google Drive to store data
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 2. Preprocess data

In [4]:
# Load data
from datasets import load_dataset
finance_dataset = load_dataset("FinanceInc/auditor_sentiment")

# Create a smaller training dataset for faster training times
from datasets import DatasetDict

small_finance_dataset = DatasetDict(
    train=finance_dataset['train'].shuffle(seed=24).select(range(500)), # for training
    val=finance_dataset['train'].shuffle(seed=24).select(range(500, 600)), #  for validation
    test=finance_dataset['train'].shuffle(seed=24).select(range(600, 700)) # for testing
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3877 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/969 [00:00<?, ? examples/s]

In [5]:
small_finance_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 500
    })
    val: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
})

In [6]:
small_finance_dataset['train'][:5]

{'sentence': ['FCC Chairman Kevin Martin said that fair play required extending the same deregulatory rules to the digital subscriber lines that telecom providers use for broadband networks .',
  'Metso Foundries Jyvaskyla Oy will discontinue production on this line by 30 September 2008 , the company said .',
  'Finnish business software group AffectoGenimap Oyj said its net profit halved to 1.2 mln euro ( $ 1.5 mln ) in the first nine months of 2006 from 2.2 mln euro ( $ 2.8 mln ) in the same period of 2005 .',
  'Finnish financial software developer Basware Oyj said today it will provide its invoice automation ( IA ) solution to an unnamed major retail company in the USA in a deal , worth more than EUR300 ,000 .',
  'According to the company , a decision in the issue will be made in the summer of 2010 , at the earliest , and in the summer of 2011 , at the latest .'],
 'label': [1, 1, 0, 2, 1]}

In [7]:
# Set distilgpt2 tokenizer
from transformers import AlbertTokenizer                         #
tokenizer = AlbertTokenizer.from_pretrained('albert-xlarge-v2')    #
print(tokenizer)
print(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

AlbertTokenizer(name_or_path='albert-xlarge-v2', vocab_size=30000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
30000


In [8]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

small_tokenized_dataset = small_finance_dataset.map(tokenize_function, batched=True, batch_size=16)
small_tokenized_dataset = small_tokenized_dataset.remove_columns(["sentence"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
small_tokenized_dataset['train'][0:3]


{'labels': tensor([1, 1, 0]),
 'input_ids': tensor([[    2, 18213,  1757,  3480,  1189,    87,    30,  1768,   418,  1390,
           8176,    14,   205,   121,    99, 15628,  7496,  1761,    20,    14,
           1888,    13, 20330,   139,  1560,    30, 18712, 13488,   275,    26,
          22831,  5540,    13,     9,     3,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [    2,   798,   656,   216,  2829,   487,    93,  1385,  2397,   531,
             13,  7452,   129,  1460, 13391,  4185,   637,    27,    48,   293,
             34,   712,   299,   570,    13,    15,    14,   237,    87,    13,
              9,     3,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0, 

In [10]:
import torch
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=16)
eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=16)

# 3. Training the model

In [11]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

In [12]:
# Define distilgpt2 as our base model:
from transformers import AlbertForSequenceClassification #
model = AlbertForSequenceClassification.from_pretrained("albert-xlarge-v2", num_labels=3)    #


model.safetensors:   0%|          | 0.00/236M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-xlarge-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
num_epochs = 3
num_training_steps = 3 * len(train_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=2048, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=2048, out_features=2048, bias=True)
                (key): Linear(in_features=2048, out_features=2048, bias=True)
                (value): Linear(in_features=2048, out_featur

In [15]:
from datasets import load_dataset, DatasetDict

small_finance_dataset = DatasetDict(
    train=finance_dataset['train'].shuffle(seed=24).select(range(300)),
    val=finance_dataset['train'].shuffle(seed=24).select(range(300, 400)),
    test=finance_dataset['train'].shuffle(seed=24).select(range(400, 500))
)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

small_tokenized_dataset = small_finance_dataset.map(tokenize_function, batched=True, batch_size=16)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
import numpy as np
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification
#model = AlbertModel.from_pretrained("albert-xlarge-v2", num_labels=3)         #
from transformers import AlbertForSequenceClassification

model = AlbertForSequenceClassification.from_pretrained("albert-xlarge-v2", num_labels=3)

arguments = TrainingArguments(
    output_dir="store_the_checkpoints_albert-xlarge_1",                           #ADJUST FOLDER !!!   #
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    lr_scheduler_type='cosine',
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    seed=224
)


early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0
)


optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-xlarge-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.904310,0.630000
2,No log,0.882421,0.630000
3,No log,0.872114,0.630000
4,No log,0.853318,0.630000
5,No log,0.848485,0.610000
6,No log,0.811847,0.610000
7,No log,0.830176,0.620000
8,No log,0.765051,0.580000
9,No log,0.788088,0.630000
10,No log,0.778841,0.630000


TrainOutput(global_step=190, training_loss=0.7971668444181743, metrics={'train_runtime': 570.6655, 'train_samples_per_second': 5.257, 'train_steps_per_second': 0.333, 'total_flos': 82064142206208.0, 'train_loss': 0.7971668444181743, 'epoch': 10.0})

In [18]:
results = trainer.predict(small_tokenized_dataset['val'])
print(results)

PredictionOutput(predictions=array([[-1.1995718 ,  1.1312273 , -0.23888847],
       [ 0.47667685, -0.46046326,  0.45354295],
       [-0.29071563, -0.00436555,  0.13278063],
       [-1.9702932 ,  1.7515806 , -0.5903261 ],
       [-0.8062418 ,  0.4970712 ,  0.1279275 ],
       [-1.8736504 ,  1.6356844 , -0.61026114],
       [-1.4601934 ,  1.435955  , -0.34070545],
       [ 0.15658762, -0.32212195,  0.42786878],
       [-0.2110864 ,  0.12672342,  0.19977738],
       [-1.792204  ,  1.8764712 , -0.6454535 ],
       [-1.355793  ,  1.2547092 , -0.22258417],
       [-1.8223007 ,  1.8135242 , -0.6331909 ],
       [-0.9722525 ,  0.65355545, -0.02354552],
       [-0.7712832 ,  0.43517795,  0.14740092],
       [-0.27294853, -0.14948882,  0.32314888],
       [-1.5210491 ,  1.5353289 , -0.36685434],
       [-2.0258365 ,  1.7239445 , -0.64231837],
       [ 0.63381505, -0.42895266,  0.44305187],
       [-0.8941155 ,  0.6631495 , -0.067325  ],
       [ 0.23155545, -0.33623672,  0.39682323],
       [-1.

In [19]:
test_str = "The company's strong quarterly earnings report resulted in a surge in stock prices, reflecting investor confidence in its future prospects."

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("store_the_checkpoints_albert-xlarge_1/checkpoint-190") #ADJUST FOLDER !!!  500/600/700-folder320'  #
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE", "NEUTRAL"][prediction])

POSITIVE


In [20]:
results = trainer.predict(small_tokenized_dataset['test'])
print(results)

PredictionOutput(predictions=array([[-1.94432259e+00,  1.71728802e+00, -5.35700917e-01],
       [-1.88330936e+00,  1.73176003e+00, -5.73867440e-01],
       [-2.23770812e-02, -3.66550833e-02,  2.68724948e-01],
       [-7.11573735e-02,  3.36796567e-02,  2.22912833e-01],
       [-1.79713881e+00,  1.64823139e+00, -5.79739869e-01],
       [-8.51648986e-01,  7.03995168e-01,  7.41270185e-02],
       [-1.84661651e+00,  1.79631352e+00, -6.59322023e-01],
       [ 6.74552083e-01, -3.14971536e-01,  4.09006625e-01],
       [ 4.63459253e-01, -3.78562570e-01,  4.96344090e-01],
       [-1.45164356e-01,  3.17985080e-02,  2.25965276e-01],
       [-1.92008531e+00,  1.69501948e+00, -6.74501419e-01],
       [-5.40900111e-01,  3.15010607e-01,  1.42865822e-01],
       [-4.86550987e-01,  5.29113054e-01,  5.42452037e-02],
       [-3.16977710e-01, -7.52890855e-02,  2.77231216e-01],
       [-1.26592731e+00,  1.31973338e+00, -1.15963906e-01],
       [ 5.18983066e-01, -5.11402011e-01,  4.50571775e-01],
       [-1.

In [21]:
small_finance_dataset['test'][5]

{'sentence': 'The company will also shut one paper machine in Finland and one in Austria , as well as two label paper machines in Finland for up to 10 months , Helsinki-based UPM said yesterday .',
 'label': 1}

In [ ]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("store_the_checkpoints_albert-xlarge_1/checkpoint-190") #ADJUST FOLDER !!!  500/600/700-folder320     #

model_inputs = tokenizer(small_tokenized_dataset['test']['sentence'], padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import os
import re
import torch
import tensorflow as tf
import tensorboard as tb

In [ ]:
import torch

path = "store_the_checkpoints_distilgpt2_1/results_vis_albert-xlarge_2"                #
layer=0
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs['input_ids'][example]:
      if token != 0:
        sp_token_position += 1
      else:
        tensor = outputs['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [small_tokenized_dataset['test']['sentence'][example],str(small_tokenized_dataset['test']['label'][example])]
    labels.append(label)
    example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Sentence','Emotion'])

  layer+=1


TensorFlow Embedding Projector API.

# 4. Analyzing new data with the model

You can also use pipeline (uncomment a cell with huggingface above):

In [ ]:
# Upload the model to the Hub
trainer.push_to_hub()

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="") # Add your saved mode in ""

sentiment_model(["The company's strong quarterly earnings report resulted in a surge in stock prices, reflecting investor confidence in its future prospects."])